In [1]:
# LOAD STATES and worm
import os
import numpy as np
import h5py

In [2]:
# Load raw file from ftps server
# select directory
bpath = os.getcwd()
#dir_dataraw= os.path.join(bpath,'data/raw/')
dir_dataraw = '/data/home/ekellbuch/andrebrown/raw/'
dir_worm_file_list = os.path.join(bpath,'results_strains/worm_files_lists/')

if not os.path.exists(dir_dataraw):
    os.makedirs(dir_dataraw)
else:
    print('Data directory exits: {} '.format(dir_dataraw))
if not os.path.exists(dir_worm_file_list):
    os.makedirs(dir_worm_file_list)
else:
    print('Output directory exits: {} '.format(dir_worm_file_list))

Data directory exits: /data/home/ekellbuch/andrebrown/raw/ 
Output directory exits: /home/ekellbuch/Projects/behworm/arhmm-celegans/results_strains/worm_files_lists/ 


In [3]:
# test if you can connect to server
# By loading single file
ftp_path='ftp://ftp.mrc-lmb.cam.ac.uk/pub/tjucikas/wormdatabase/results-12-06-08/Laura%20Grundy/gene_NA/allele_NA/'
file_path = 'AQ2947/on_food/XX/no_wait/L/tracker_1/2012-03-08___16_27_17/'
file_name_raw  = '483 AQ2947 no wait on food R_2012_03_08__16_27_17___1___9_features.mat'

In [4]:
# check filename is exists in ftps
import urllib.request
# check if directory exists
if not os.path.exists(os.path.join(dir_dataraw,file_path)):
    os.makedirs(os.path.join(dir_dataraw,file_path))
# check if file exists in ftp
# check if file loaded or need to load
if os.path.exists(os.path.join(dir_dataraw,file_path,file_name_raw)):
    print('File available\nRaw file: {} in directory {}'.format(file_name_raw,dir_dataraw))
else:
    print('Loading file\nRaw file: {} in directory {}'.format(file_name_raw,dir_dataraw))
    urllib.request.urlretrieve(os.path.join(ftp_path,file_path,file_name_raw), 
                               os.path.join(dir_dataraw,file_path,file_name_raw))

File available
Raw file: 483 AQ2947 no wait on food R_2012_03_08__16_27_17___1___9_features.mat in directory /data/home/ekellbuch/andrebrown/raw/


In [8]:
# name of files in script
ftp_files = 'andre_ftp_matfiles.txt'
if not os.path.exists(ftp_files):
    print('Error : Download filenames not available\n')
    print('Data is available upon request\n')

In [9]:
strain_types = ['AQ2947','CB4852','CB4853','ED3017','ED3021','ED3054','N2']

In [10]:
# parse through files N2
strain_type = strain_types[6]
data_files = []
with open(ftp_files) as f:
    for line in f :
        line = line.rstrip('\n')
        if not(strain_type in line):
            continue
        # check if file available:
        if os.path.exists(dir_dataraw +line):
            print('File exists\n{}\n'.format(dir_dataraw +line))
            # check if ftype:
            if strain_type in line:
                data_files.append(line)           
        else:
            print('Not loaded\n'+line)
            if not os.path.exists(os.path.join(dir_dataraw,os.path.dirname(line))):
                print('Creating directory\n')
                os.makedirs(os.path.join(dir_dataraw,os.path.dirname(line)))
            urllib.request.urlretrieve(os.path.join(ftp_path,line),os.path.join(dir_dataraw,line))
num_files = len(data_files)

print('Loaded {} datasets from {} strain'.format(num_files,strain_type)) 

File exists
/data/home/ekellbuch/andrebrown/raw/N2/on_food/XX/no_wait/R/tracker_8/2012-03-30___11_29_18/N2 no wait on food L_2012_03_30__11_29_18___8___1_features.mat

File exists
/data/home/ekellbuch/andrebrown/raw/N2/on_food/XX/no_wait/R/tracker_7/2012-03-30___11_28_58/N2 no wait on food L_2012_03_30__11_28_58___7___1_features.mat

File exists
/data/home/ekellbuch/andrebrown/raw/N2/on_food/XX/no_wait/R/tracker_7/2012-02-22___11_55_01/N2 no wait on food L_2012_02_22__11_55_01___7___5_features.mat

File exists
/data/home/ekellbuch/andrebrown/raw/N2/on_food/XX/no_wait/R/tracker_6/2012-03-30___11_25_12/N2 no wait on food L_2012_03_30__11_25_12___6___1_features.mat

File exists
/data/home/ekellbuch/andrebrown/raw/N2/on_food/XX/no_wait/R/tracker_5/2012-04-03___12_17_04/N2 no wait on food_2012_04_03__12_17_04___5___1_features.mat

File exists
/data/home/ekellbuch/andrebrown/raw/N2/on_food/XX/no_wait/R/tracker_5/2012-03-30___11_28_20/N2 no wait on food L_2012_03_30__11_28_20___5___1_features

In [11]:
# Worms must match the same number of frames per second
frames_per_second = np.zeros(shape=(num_files,))
for ii, myfile in enumerate(data_files):
    try:
        f = h5py.File(os.path.join(dir_dataraw,myfile),'r')
    # check worms fps
        frames_per_second[ii] = f['info']['video']['resolution']['fps'][0][0]
    except:
        print('cannot load it')
        pass

cannot load it
cannot load it
cannot load it


In [12]:
# Select datasets sharing most common fps
from collections import Counter
fps_vals = Counter(frames_per_second)
print('Keeping (fps,numfiles) {}'.format(fps_vals.most_common(1)))
fps_val  = fps_vals.most_common(1)[0][0]
data_files_fps = np.asarray(data_files)[np.where(frames_per_second == fps_val)[0]]

Keeping (fps,numfiles) [(30.03003003003003, 596)]


In [13]:
# take files that have at least 10k non-NAN values
min_non_nan = int(1e4)
remove_file = []
for ii, myfile in enumerate(data_files_fps):
    f = h5py.File(os.path.join(dir_dataraw,myfile),'r')
    if np.count_nonzero(~np.isnan(f['worm']['posture']['skeleton']['x'][()])) < min_non_nan:
        print('Removing file {} : {}'.format(ii,myfile))
        remove_file.append(ii)

Removing file 86 : N2/on_food/XX/30m_wait/R/tracker_7/2011-03-17___11_33_44/N2 on food L_2011_03_17__11_33_44___7___2_features.mat
Removing file 218 : N2/on_food/XX/30m_wait/R/tracker_4/2011-05-25___14_41_00/N2 Robyn on food L_2011_05_25__14_41___3___2_features.mat
Removing file 219 : N2/on_food/XX/30m_wait/R/tracker_4/2011-05-12___11_41_00/N2 on food L_2011_05_12__11_41___3___4_features.mat


In [14]:
# Original paper with strain_type = 'N2'
#np.savez(dir_worm_file_list+'worm_file_list.npz',data_files_fps=data_files_fps)
if True:
    fname_out = os.path.join(dir_worm_file_list,'worm_file_list_'+strain_type+'.npz')
    if not os.path.isfile(fname_out):
        np.savez(fname_out,data_files_fps=data_files_fps,remove_file=remove_file,fps_val=fps_val)
    else:
        print('File:\n{}\nalready exists'.format(fname_out))

File:
/home/ekellbuch/Projects/behworm/arhmm-celegans/results_strains/worm_files_lists/worm_file_list_N2.npz
already exists
